In [ ]:
!git clone https://github.com/MobileRoboticistsW21/PHD-object-traking.git
%cd PHD-object-traking/

In [ ]:
%cd PHD-object-traking/

In [ ]:
!git checkout plot_in_notebook   # can change this to main if branch is merged 
!git pull

In [ ]:
%cd /content/PHD-object-traking/
!bash setup.sh
!mkdir build
%cd build

In [ ]:
!cmake ..
!make phd_json_dump

In [ ]:
!./phd_json_dump
# enter this: ../data/first_20_frame_optical_res.json

In [ ]:
#TODO:
#call the visualization here:

# motion model
# plot in python
# look into weights
# 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
with open("/content/PHD-object-traking/build/tracks_dump.json", encoding='UTF-8', errors='ignore') as inputFile:
    data = json.load(inputFile)

In [ ]:
for i in range(len(data)):             # number of frames
    n = len(data[i]['bb'])             # number of particles
    bbs = data[i]['bb']
    weights = data[i]['weights']
    print("frame ", i)
    print(weights)
    #flows = data[i]['flows']
    plt.clf();
    plt.xlim(0, 540);
    plt.ylim(0, 960);
    
    ax = plt.axes()
    for bb in bbs:
        rect = patches.Rectangle((bb[0] - bb[2] / 2, bb[1] - bb[3] / 2), bb[2], bb[3], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
    plt.pause(0.000001);


In [ ]:
from python_utils.id_assigner import assign_ids_postfact
data = assign_ids_postfact(data)

with open('/content/PHD-object-traking/build/run_phd_output_extract_with_ids.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)